In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from dataset import Data
from callback import Save

from transformations import chargenet_trafo

In [2]:
data = Data(dir='/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols')

In [9]:
charge_input = tf.keras.Input(shape=(1,))
params_input = tf.keras.Input(shape=(len(data.labels),))
t = chargenet_trafo(labels=data.labels)

h = t(charge_input, params_input)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
#h = tf.keras.layers.Dense(512, activation="relu")(h)
#h = tf.keras.layers.Dropout(0.001)(h)
#h = tf.keras.layers.Dense(256, activation="relu")(h)
#h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(h)

charge_net = tf.keras.Model(inputs=[charge_input, params_input], outputs=outputs)
charge_net.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
chargenet_trafo_1 (chargenet_tr (None, 13)           0           input_3[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 32)           448         chargenet_trafo_1[0][0]          
____________________________________________________________________________________________

In [14]:
optimizer = tf.keras.optimizers.Adam(0.00001)
#optimizer = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.8)
#optimizer = tf.keras.optimizers.RMSprop()
charge_net.compile(loss='binary_crossentropy', optimizer=optimizer)

In [10]:
train_id = 'ChargeNet_' + 'L3_BN+DO' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [11]:
callbacks = []
callbacks.append(Save(save_every=25, path_template='models/'+train_id+'/epoch_%i', ))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='logs/'+train_id, histogram_freq=1))

In [12]:
train_data, test_data = data.get_chargenet_data(train_batch_size=128,
                                                 test_batch_size=4096,
                                                )

In [ ]:
hist = charge_net.fit(train_data, epochs=1000, verbose=2, callbacks=callbacks, validation_data=test_data,
                     initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 1/1000
19447/19447 - 129s - loss: 0.6698 - val_loss: 0.0000e+00
Epoch 2/1000
19447/19447 - 123s - loss: 0.6298 - val_loss: 0.6318
Epoch 3/1000
19447/19447 - 115s - loss: 0.6115 - val_loss: 0.6198
Epoch 4/1000
19447/19447 - 118s - loss: 0.5999 - val_loss: 0.6162
Epoch 5/1000
19447/19447 - 116s - loss: 0.5919 - val_loss: 0.6079
Epoch 6/1000
19447/19447 - 115s - loss: 0.5862 - val_loss: 0.6088
Epoch 7/1000
19447/19447 - 115s - loss: 0.5817 - val_loss: 0.6059
Epoch 8/1000
19447/19447 - 118s - loss: 0.5780 - val_loss: 0.6035
Epoch 9/1000
19447/19447 - 118s - loss: 0.5751 - val_loss: 0.5995
Epoch 10/1000
19447/19447 - 125s - loss: 0.5725 - val_loss: 0.5942
Epoch 11/1000
19447/19447 - 117s - loss: 0.5703 - val_loss: 0.5892
Epoch 12/1000
19447/19447 - 115s - loss: 0.5683 - val_loss: 0.5995
Epoch 13/1000
19447/19447 - 114s - loss: 0.5665 - val_loss: 0.5901
Epoch 14/1000
19447/19447 - 115s - loss: 0.5649 - val_loss: 0.5932
Epoch 15/1000
19447/19447 - 115s - loss: 0.5635 - val_loss: 0.5893


In [ ]:
# best valid loss L7 0.53....52
# L3: 0.48 ish